In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/training-set/train_age_dataset.csv


In [2]:
df=pd.read_csv('/kaggle/input/training-set/train_age_dataset.csv')
df.head(2)

,Unnamed: 0,userId,tier,gender,following_rate,followers_avg_age,following_avg_age,max_repetitive_punc,num_of_hashtags_per_action,emoji_count_per_action,...,content_views,num_of_comments,weekends_trails_watched_per_day,weekdays_trails_watched_per_day,slot1_trails_watched_per_day,slot2_trails_watched_per_day,slot3_trails_watched_per_day,slot4_trails_watched_per_day,avgt2,age_group
0,265153,48958844,2,1,0.0,0.0,0.0,0,0.0,0.0,...,0.20000,0.0,0.041667,0.025000,0.0,0.000000,0.175,0.033333,0.0,1
1,405231,51100441,2,2,0.0,0.0,0.0,0,0.0,0.0,...,0.09322,0.0,0.012712,0.018644,0.0,0.084746,0.000,0.033898,82.5,2


In [3]:
df.drop(['Unnamed: 0','userId'],axis=1,inplace=True)

In [4]:
from xgboost import XGBClassifier
from sklearn import metrics
from sklearn.model_selection import train_test_split

In [5]:
X=df.drop(['age_group'],axis=1)
y=df['age_group']

In [6]:
train_x,test_x,train_y,test_y=train_test_split(X,y,test_size=.06)

In [7]:
from sklearn.utils.class_weight import compute_class_weight
classes = np.unique(train_y)
weights = compute_class_weight(class_weight='balanced', classes=classes, y=train_y)
class_weights = dict(zip(classes, weights))
class_weights

{1: 0.39641013608634446,
 2: 2.061253050093297,
 3: 2.020755281163703,
 4: 2.0106407182484816}

In [ ]:
import sklearn
from sklearn import *
xgb_clf = XGBClassifier(tree_method = 'gpu_hist', gpu_id=0, objective = "multi:softmax",num_class = 4, class_weights=class_weights)

# Create parameter grid
parameters = {"learning_rate": [0.001],"max_depth": [14, 21],"n_estimators": [750, 1500],"reg_lambda": [ 2.5, 5]}

from sklearn.model_selection import RandomizedSearchCV
scorer = metrics.make_scorer(sklearn.metrics.f1_score, average = 'weighted')
#gs_svc = GridSearchCV(estimator=svc_clf,param_grid=param_grid,scoring=scorer,cv=5)
# Create RandomizedSearchCV Object
xgb_rscv = RandomizedSearchCV(xgb_clf, param_distributions = parameters, scoring = scorer,cv=2)

# Fit the model
model_xgboost = xgb_rscv.fit(train_x, train_y)

# Model best estimators
print("Learning Rate: ", model_xgboost.best_estimator_.get_params()["learning_rate"])
print("Lamda: ", model_xgboost.best_estimator_.get_params()["reg_lambda"])
print("Max Depth: ", model_xgboost.best_estimator_.get_params()["max_depth"])
print("Subsample: ", model_xgboost.best_estimator_.get_params()["subsample"])


print("Number of Trees: ", model_xgboost.best_estimator_.get_params()["n_estimators"])

/opt/conda/lib/python3.7/site-packages/sklearn/model_selection/_search.py:282: UserWarning: The total space of parameters 8 is smaller than n_iter=10. Running 8 iterations. For exhaustive searches, use GridSearchCV.
  % (grid_size, self.n_iter, grid_size), UserWarning)
